In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../helper_functions')
sys.path.append('../objects')
# sys.path.append('..')

import copy
import time
import dill
import numpy as np
import scipy.io as io
import mat73
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation as R
import plotly.graph_objects as go

%matplotlib inline
# %matplotlib qt

In [3]:
from prefixes import remove_prefix, add_prefix
from conversions import rho24sig, convert2mainSI

from ElectricObject import ElectricObject

from Worm import Worm
from Worm_SmallSpherical import SmallSphericalWorm

from Fish import Fish
from FishGeneration import FishGeneration
from Fish_IceCreamCone import IceCreamConeFish
from Fish_Capsule import CapsuleFish


from Boundaries import *
from Aquarium import Aquarium
from Aquarium_SinglePlane import SinglePlaneAquarium

from assertion_tests import run_tests
run_tests()

'Success!'

## Load and process the EOD waveform

In [4]:
# FISH_EOD_ID = 0
# sampling_rate = (2.5, 'M')
# full_eod = io.loadmat('../../eods/eod_fit.mat')['fitted_eod'].flatten()

# # erase the 0-padding around the eod for efficiency
# max_eod = np.abs(full_eod).max()
# alt_0 = 1e-2
# good_id_bounds = np.where(np.abs(full_eod) > max_eod * alt_0)[0]
# full_eod = full_eod[good_id_bounds[0]:good_id_bounds[-1]]
# full_eod-= full_eod[0]
# full_eod = full_eod[:1000]
# full_eod.shape

In [5]:
FISH_EOD_ID = 2
# sampling_rate = (1.428469395043211, 'M')
sampling_rate = (2.5, 'M')
full_eod = mat73.loadmat(f'../../eods/20231022_eod_{FISH_EOD_ID}.mat')['eod_fish'].flatten()

# # erase the 0-padding around the eod for efficiency
# max_eod = np.abs(full_eod).max()
# alt_0 = 1e-2
# good_id_bounds = np.where(np.abs(full_eod) > max_eod * alt_0)[0]
# full_eod = full_eod[good_id_bounds[0]:good_id_bounds[-1]]
# full_eod-= full_eod[0]
full_eod.shape

(2501,)

## Create the Fish object

In [27]:
# define the base fish
base_fish = CapsuleFish(
    receptors_locations=np.array([[11e-2-0.02, 0.007, 0]]).reshape(1,3),
    receptors_normals=np.array([[0, 1, 0]]).reshape(1,3),
    # point_currents_magnitudes=np.array([]).reshape(0),
    # point_currents_locations=np.array([]).reshape(0,3),
    receptor_filters=np.array([]).reshape(0,full_eod.shape[0]),
    eod_waveform=full_eod,
    skin_resistivity=3 * 1e3 * (1e-2)**2,
    sampling_rate=sampling_rate,  # (2.5, "M"),  # MHz,
    eod_delay=(0,), # s,
    nose_position=[0,0,0],
    fish_length=(11, 'c'),
    angle_yaw=(0, 'deg'),
    angle_pitch=(0, 'deg'),
    angle_roll=(0, 'deg'),
    relative_bend_locations_percentage=np.array([]),
    relative_bend_angle_lateral=(np.array([]),"deg"),
    relative_bend_angle_dorso_ventral =(np.array([]),"deg"),
    point_currents_range_percentage=dict(start=0, end=100),
    N_point_currents=101,
    point_currents_magnitude_scale=1,
    receptors_init=None,
    vertical_semi_axis=(1.4, 'c'),
    lateral_semi_axis=(0.7, 'c'),
    rostrocaudal_semi_axis=0,#None,
    rostrocaudal_semi_axis_tail=0,#(3, 'c'),
)

dill.dump(base_fish,  open(f'../data/baseFish/data_231022_base_fish_ID{FISH_EOD_ID}_single_receptor.pkl', 'wb'), protocol=4)
# base_fish = dill.load(open(f'../data/baseFish/data_231022_base_fish_ID{FISH_EOD_ID}_single_receptor.pkl', 'rb'))

Success!


In [28]:
# base_fish.visualize_scatter(show_normals=2, show_point_currents=3)
# print(base_fish.get_receptors_locations().shape)

In [29]:
# Create Data Params Dict
data_params_dict = {}
data_params_dict['save_file_name']                  = f'data_231022_SingleFish_EODid{FISH_EOD_ID}_ManyRC'
# aquarium properties
data_params_dict['water_conductivities']            = [0.01]  # np.array([100, 300, 600]) * 1e-6 / 1e-2
data_params_dict['boundary_displacements']          = [[-1e9]]
data_params_dict['boundary_normals']                =  [[0,0,1]]
# fish properties
data_params_dict['fish_obj']                        = base_fish
data_params_dict['fish_bend_angle_lateral']         = [[0]]  #[[x] for x in np.linspace(-60,60,25)]
data_params_dict['fish_bend_angle_dorso_ventral']   = [[0]]  #[[0]]
data_params_dict['fish_bend_location_percentages']  = [[0]]  #[[60]]
data_params_dict['fish_yaw']                        = [0]
data_params_dict['fish_pitch']                      = [0]
data_params_dict['fish_roll']                       = [0]
# worm properties
data_params_dict['worm_resistances']                = (np.exp(np.linspace(np.log(2), np.log(100), 100)) * 1e3)[20:80][::4]
data_params_dict['worm_capacitances']               = (np.exp(np.linspace(np.log(2), np.log(1000), 100)) * 1e-10)[0:45][::3]
data_params_dict['worm_radii']                      = np.array([5]) * 1e-3
data_params_dict['worm_position_xs']                = np.array([-20]) * 1e-3
data_params_dict['worm_position_ys']                = np.array([5]) * 1e-3 + base_fish.lat_ax
data_params_dict['worm_position_zs']                = np.array([0]) * 1e-3
# save params file
dill.dump(data_params_dict, open('../data/params/' + data_params_dict['save_file_name'] + '_params_dict.pkl', 'wb'), protocol=4)

# Modify previous parameters for re-runs

In [ ]:
# fish = data_params_dict['fish_obj']
data_params_dict = dill.load(open('../data/params/data_220610_Large_dataset_with_receptors_responses_params_dict.pkl','rb'))

In [ ]:
data_params_dict

In [ ]:
# SECOND (LARGER STD) GAUSSIAN RANDOM IN R & C WITH MEAN AROUND MOST VARIABLE POINT IN FEATURE SPACE
resistances = np.sort(np.abs(np.random.randn(200) * data_params_dict['worm_resistances'][::8][9-3:9+3].std() * 2 + data_params_dict['worm_resistances'][::8][9-3:9+3].mean()))
capacitances = np.sort(np.abs(np.random.randn(200) * data_params_dict['worm_capacitances'][::8][12-3:12+3].std() * 2 + data_params_dict['worm_capacitances'][::8][12-3:12+3].mean()))

data_params_dict['worm_resistances'] = resistances
data_params_dict['worm_capacitances'] = capacitances
data_params_dict['fish_obj'] = base_fish
data_params_dict['worm_position_xs'] = [-0.03]
data_params_dict['save_file_name'] = 'data_221220_PCA_of_RC_effects_Gauss_Wide_Rand_Dist'

In [ ]:
# plt.plot(resistances)
plt.plot(capacitances)

In [ ]:
data_params_dict

In [ ]:
dill.dump(data_params_dict, open('../data/params/'+data_params_dict['save_file_name'] + '_params_dict.pkl', 'wb'), protocol=4)